# API-REST: AEMET OPEN DATA 

En este notebook veremos otro ejemplo de uso de la api open data de AEMET. En este caso obtendremos parámetros medidos por una estación meteorológica.

In [1]:
import requests
import datetime
import time

In [2]:
import urllib3
urllib3.disable_warnings()

In [3]:
# Cargamos la api key 
# Get API KEY in AEMET: https://opendata.aemet.es/centrodedescargas/inicio
api_key = open("../../apikey-aemet.txt").read().rstrip()
querystring = {"api_key": api_key}

In [4]:
def generate_chunks(start_date, final_date, step):
    day_step = datetime.timedelta(days=1)
    
    next_date = start_date + step
    
    while next_date < final_date:
        yield (start_date, next_date)
        start_date = next_date + day_step
        next_date = start_date + step
    
    yield (start_date, final_date)

In [5]:
start_date = datetime.datetime(2017, 1, 1, 0, 0, 0)
final_date = datetime.datetime(2017, 3, 15, 0, 0, 0)
step = datetime.timedelta(days=30)
list(generate_chunks(start_date, final_date, step))

[(datetime.datetime(2017, 1, 1, 0, 0), datetime.datetime(2017, 1, 31, 0, 0)),
 (datetime.datetime(2017, 2, 1, 0, 0), datetime.datetime(2017, 3, 3, 0, 0)),
 (datetime.datetime(2017, 3, 4, 0, 0), datetime.datetime(2017, 3, 15, 0, 0))]

In [6]:
def get_daily_data_url(start, end, station):
    url = ("https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/"
           "fechaini/{start}/fechafin/{end}/estacion/{station}".format(
               start=start_date.strftime('%Y-%m-%dT%H:%M:%SUTC'),
               end=final_date.strftime('%Y-%m-%dT%H:%M:%SUTC'),
               station=station)
          )
    return url

In [7]:
def make_request(url):
    request_again = True
    sleep_secs = 1
    while request_again:
        r = requests.get(url, params=querystring, verify=False)
        print(r)
        if r.status_code == requests.codes.OK:
            request_again = False
            data_url = r.json()['datos']
            r_data = requests.get(data_url, params=querystring, verify=False)
            raw_data = r_data.json()
        else:
            print(f"failed, sleep {sleep_secs}s and try again")
            time.sleep(sleep_secs)
            sleep_secs += 1
        
    return raw_data

In [8]:
("https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/"
       "fechaini/2017-01-01T00%3A00%3A00UTC/fechafin/2017-01-30T00%3A00%3A00UTC/estacion/8025")

'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2017-01-01T00%3A00%3A00UTC/fechafin/2017-01-30T00%3A00%3A00UTC/estacion/8025'

In [9]:
start_date = datetime.datetime(2015, 1, 1, 0, 0, 0)
final_date = datetime.datetime(2017, 12, 31, 0, 0, 0)
step = datetime.timedelta(days=30)

chunks = generate_chunks(start_date, final_date, step)

station='8025,8019,08370,7247X'

data = []

for start_date, final_date in chunks:
    url = get_daily_data_url(start_date, final_date, station)
    print(url)
    req_data = make_request(url)
    data  = data + req_data
    
    time.sleep(1.5)

# Convert to numeric
for d in data:
    for param in ['prec', 'presMax', 'presMin', 'racha', 'sol', 'tmax', 'tmed', 'tmin', 'velmedia', 'altitud', 'dir']:
        try:
            d[param] = float(d[param].replace(',', '.'))
        except:
            d[param] = None

https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2015-01-01T00:00:00UTC/fechafin/2015-01-31T00:00:00UTC/estacion/8025,8019,08370,7247X
<Response [200]>
https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2015-02-01T00:00:00UTC/fechafin/2015-03-03T00:00:00UTC/estacion/8025,8019,08370,7247X
<Response [200]>
https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2015-03-04T00:00:00UTC/fechafin/2015-04-03T00:00:00UTC/estacion/8025,8019,08370,7247X
<Response [200]>
https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2015-04-04T00:00:00UTC/fechafin/2015-05-04T00:00:00UTC/estacion/8025,8019,08370,7247X
<Response [200]>
https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2015-05-05T00:00:00UTC/fechafin/2015-06-04T00:00:00UTC/estacion/8025,8019,08370,7247X
<Response [200]>
https://opendata.aemet.es/opendata/api/valores/climatologico

In [10]:
len(data)

3262

In [11]:
data[0]

{'altitud': 81.0,
 'dir': 14.0,
 'fecha': '2015-01-01',
 'horaPresMax': 'Varias',
 'horaPresMin': '00',
 'horaracha': '14:00',
 'horatmax': '13:20',
 'horatmin': '07:30',
 'indicativo': '8025',
 'nombre': 'ALICANTE/ALACANT',
 'prec': 0.0,
 'presMax': 1027.2,
 'presMin': 1019.8,
 'provincia': 'ALICANTE',
 'racha': 4.7,
 'sol': 8.6,
 'tmax': 16.0,
 'tmed': 9.6,
 'tmin': 3.2,
 'velmedia': 1.4}

In [12]:
import pandas as pd

In [13]:
df = pd.DataFrame(data)

In [14]:
df.head()

,altitud,dir,fecha,horaPresMax,horaPresMin,horaracha,horatmax,horatmin,indicativo,nombre,prec,presMax,presMin,provincia,racha,sol,tmax,tmed,tmin,velmedia
0,81.0,14.0,2015-01-01,Varias,00,14:00,13:20,07:30,8025,ALICANTE/ALACANT,0.0,1027.2,1019.8,ALICANTE,4.7,8.6,16.0,9.6,3.2,1.4
1,575.0,5.0,2015-01-01,NaN,NaN,06:00,15:00,06:50,7247X,PINOSO,0.0,NaN,NaN,ALICANTE,4.4,NaN,12.1,3.7,-4.7,1.1
2,43.0,NaN,2015-01-01,Varias,00,NaN,12:46,21:30,8019,ALICANTE-ELCHE AEROPUERTO,0.0,1033.5,1026.0,ALICANTE,NaN,6.8,15.9,10.8,5.7,1.9
3,81.0,19.0,2015-01-02,11,04,14:00,14:00,07:20,8025,ALICANTE/ALACANT,0.0,1028.8,1025.3,ALICANTE,4.2,8.8,18.1,10.0,2.0,1.1
4,575.0,99.0,2015-01-02,NaN,NaN,Varias,14:30,06:50,7247X,PINOSO,0.0,NaN,NaN,ALICANTE,2.8,NaN,16.2,5.8,-4.6,1.1


In [15]:
interesting_cols = [
    "dir",
    "fecha",
    # "horaPresMax",
    # "horaPresMin",
    # "horaracha",
    # "horatmax",
    # "horatmin",
    "indicativo",
    "nombre",
    "prec",
    "presMax",
    "presMin",
    "provincia",
    "racha",
    "sol",
    "tmax",
    "tmed",
    "tmin",
    "velmedia"
]

In [16]:
CSV_FILE = '../data/alicante_climate_AEMET.csv'

In [17]:
df.to_csv(CSV_FILE, index=False, sep='\t', columns=interesting_cols)

In [18]:
with open(CSV_FILE) as f:
    lines = f.readlines()
    now = time.strftime("%c")
    final = ["created on: " + now + '\n'*2] + lines

with open(CSV_FILE, mode='w') as f:
    f.writelines(final)

In [19]:
df = df.loc[df['indicativo'] == '8025']

In [20]:
CSV_FILE = '../data/alicante_city_climate_AEMET.csv'

In [21]:
df.to_csv(CSV_FILE, index=False, sep='\t', columns=interesting_cols)

In [22]:
with open(CSV_FILE) as f:
    lines = f.readlines()
    now = time.strftime("%c")
    final = ["created on: " + now + '\n'*2] + lines

with open(CSV_FILE, mode='w') as f:
    f.writelines(final)